In [ ]:
#| hide
from xcube.utils import *

# xcube

> Extreme Multi-Label Text Classification

This is under development.

This file will become your README and also the index of your documentation.

## Install

Install using:

```sh
pip install xcube
```

or
```sh
conda install xcube
```

## How to use

Fill me in please! Don't forget code examples:

## Acknowledgement

This repository is my attempt to create Extreme Multi-Label Text Classifiers using Language Model Fine-Tuning as proposed by [Jeremy Howard](https://jeremy.fast.ai) and [Sebastian Ruder](https://www.ruder.io) in [ULMFit](https://arxiv.org/pdf/1801.06146v5.pdf). I am also heavily influenced by the [fast.ai's](https://fast.ai) course [Practical Deep Learning for Coders](https://course.fast.ai/) and the excellent library [fastai](https://github.com/fastai/fastai). I have adopted the style of coding from [fastai](https://github.com/fastai/fastai) using the jupyter based dev environment [nbdev](https://nbdev.fast.ai/). Since this is one of my fast attempt to create a full fledged python library, I have at times replicated implementations from fastai with some modifications. A big thanks to Jeremy and his team from [fast.ai](https://fast.ai) for everything they have been doing to make AI accessible to everyone.